<hr />

### NoteBookの見方
コード以外の情報 
<span >白/黒での記載は速習Symbol</span>  
<span style="color:red">赤色での記載は補足情報</span>  
<span >コード内で🌟マークがある場所は自分の情報に書き換えが必要</span>

<hr />

<span style="color:red">

# 環境構築
</span>
<span style="color:red">

## 1.Symbol SDKの読み込み </span>

In [ ]:
(script = document.createElement("script")).src = "https://xembook.github.io/nem2-browserify/symbol-sdk-pack-2.0.3.js";
document.getElementsByTagName("head")[0].appendChild(script);

<span style="color:red">

## 2.Symbol用の共通設定 </span>

In [ ]:
NODE = 'https://sym-test-03.opening-line.jp:3001';
sym = require("/node_modules/symbol-sdk");
repo = new sym.RepositoryFactoryHttp(NODE);
txRepo = repo.createTransactionRepository();
mosaicRepo = repo.createMosaicRepository();
accountRepo = repo.createAccountRepository();
(async () => {
  networkType = await repo.getNetworkType().toPromise();
  generationHash = await repo.getGenerationHash().toPromise();
  epochAdjustment = await repo.getEpochAdjustment().toPromise();
})();

function clog(signedTx){
    console.log(NODE + "/transactionStatus/" + signedTx.hash);
    console.log(NODE + "/transactions/confirmed/" + signedTx.hash);
    console.log("https://symbol.fyi/transactions/" + signedTx.hash);
    console.log("https://testnet.symbol.fyi/transactions/" + signedTx.hash);
}

<span style="color:red">

## 3.aliceアカウントのリストア </span>

In [ ]:
alice = sym.Account.createFromPrivateKey(
    "1E9139CC1580B4AED6A1FE110085281D4982ED0D89CE07F3380EB83069B1****", //🌟ここに3章で作成した秘密鍵を入力
    networkType
);

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229624965-eaf6ae6e-7a61-4cf4-9ca9-1bdfad5cafbf.png">

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/233480494-04999ba0-766d-437f-8657-d28933e90918.png">

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229624993-7527cdd1-7919-44b9-9c6f-1f5b3e4d1310.png">

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229625002-455e6099-9b95-431d-8686-d09f91b9a16d.png">

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/233480526-8aff16da-8962-47d0-beb1-9f50852497c0.png">

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229625022-c67dc00f-d002-4120-8a14-fe885a95f670.png">

<hr />

# 11.制限

アカウントに対する制限とモザイクのグローバル制限についての方法を紹介します。
本章では、既存アカウントの権限を制限してしまうので、使い捨てのアカウントを新規に作成してお試しください。

In [ ]:
//使い捨てアカウントCarolの生成
carol = sym.Account.generateNewAccount(networkType);
console.log(carol.address);
//FAUCET URL出力
console.log("https://testnet.symbol.tools/?recipient=" + carol.address.plain() +"&amount=100");

## 11.1 アカウント制限

### 指定アドレスからの受信制限・指定アドレスへの送信制限

In [ ]:
bob = sym.Account.generateNewAccount(networkType);
tx = sym.AccountRestrictionTransaction.createAddressRestrictionModificationTransaction(
  sym.Deadline.create(epochAdjustment),
  sym.AddressRestrictionFlag.BlockOutgoingAddress, //アドレス制限フラグ　速習SymbolではBlockIncomingAddressだが勉強会の都合上BlockOutgoingAddressに変更
  [bob.address],//設定アドレス
  [],　　　　　　//解除アドレス
  networkType
).setMaxFee(100);
signedTx = carol.sign(tx,generationHash);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

AddressRestrictionFlagについては以下の通りです。
```js
{1: 'AllowIncomingAddress', 16385: 'AllowOutgoingAddress', 32769: 'BlockIncomingAddress', 49153: 'BlockOutgoingAddress'}
```

AddressRestrictionFlagにはBlockIncomingAddressのほか、上記のようなフラグが使用できます。
- AllowIncomingAddress：指定アドレスからのみ受信許可
- AllowOutgoingAddress：指定アドレス宛のみ送信許可
- BlockIncomingAddress：指定アドレスからの受信受拒否
- BlockOutgoingAddress：指定アドレス宛への送信禁止

<span style="color:red">carolからbobへのトランザクション（失敗することの確認）</span>  

In [ ]:
trTx = sym.TransferTransaction.create(
    sym.Deadline.create(epochAdjustment),
    bob.address, 
    [new sym.Mosaic(new sym.MosaicId("72C0212E67A08BCE"), sym.UInt64.fromUint(1))],
    sym.PlainMessage.create(""),
    networkType
  ).setMaxFee(100);
signedTx = carol.sign(trTx,generationHash);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

<span style="color:red">carolからaliceへのトランザクション（成功することの確認）</span>  

In [ ]:
trTx = sym.TransferTransaction.create(
    sym.Deadline.create(epochAdjustment),
    alice.address, 
    [new sym.Mosaic(new sym.MosaicId("72C0212E67A08BCE"), sym.UInt64.fromUint(1))],
    sym.PlainMessage.create(""),
    networkType
  ).setMaxFee(100);
signedTx = carol.sign(trTx,generationHash);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

<span style="color:red">bobへの送信制限を解除</span>  

In [ ]:
tx = sym.AccountRestrictionTransaction.createAddressRestrictionModificationTransaction(
    sym.Deadline.create(epochAdjustment),
    sym.AddressRestrictionFlag.BlockOutgoingAddress, //アドレス制限フラグ　速習SymbolではBlockIncomingAddressだが勉強会の都合上BlockOutgoingAddressに変更
    [],//設定アドレス
    [bob.address],//解除アドレス
    networkType
  ).setMaxFee(100);
  signedTx = carol.sign(tx,generationHash);
  await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

<span style="color:red">再度carolからbobへのトランザクション（制限がなくなり成功することの確認）</span>  

In [ ]:
trTx = sym.TransferTransaction.create(
    sym.Deadline.create(epochAdjustment),
    bob.address, 
    [new sym.Mosaic(new sym.MosaicId("72C0212E67A08BCE"), sym.UInt64.fromUint(1))],
    sym.PlainMessage.create(""),
    networkType
  ).setMaxFee(100);
signedTx = carol.sign(trTx,generationHash);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

### 指定モザイクの受信制限

In [ ]:
mosaicId = new sym.MosaicId("72C0212E67A08BCE"); //テストネット XYM
tx = sym.AccountRestrictionTransaction.createMosaicRestrictionModificationTransaction(
  sym.Deadline.create(epochAdjustment),
  sym.MosaicRestrictionFlag.AllowMosaic, //モザイク制限フラグ 　速習SymbolではBlockMosaicだが勉強会の都合上AllowMosaicに変更
  [mosaicId],//設定モザイク
  [],//解除モザイク
  networkType
).setMaxFee(100);
signedTx = carol.sign(tx,generationHash);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

MosaicRestrictionFlagについては以下の通りです。
```js
{2: 'AllowMosaic', 32770: 'BlockMosaic'}
```

- AllowMosaic：指定モザイクを含むトランザクションのみ受信許可
- BlockMosaic：指定モザイクを含むトランザクションを受信拒否

モザイク送信の制限機能はありません。
また、後述するモザイクのふるまいを制限するグローバルモザイク制限と混同しないようにご注意ください。

<span style="color:red">aliceからcarolへのXYMの送信（成功することの確認）</span>  

In [ ]:
trTx = sym.TransferTransaction.create(
    sym.Deadline.create(epochAdjustment),
    carol.address, 
    [new sym.Mosaic(new sym.MosaicId("72C0212E67A08BCE"), sym.UInt64.fromUint(1))],
    sym.PlainMessage.create(""),
    networkType
  ).setMaxFee(100);
signedTx = alice.sign(trTx,generationHash);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

<span style="color:red">aliceからcarolへのhossiiii.tomatoの送信（失敗することの確認）</span>  

In [ ]:
trTx = sym.TransferTransaction.create(
    sym.Deadline.create(epochAdjustment),
    carol.address, 
    [new sym.Mosaic(new sym.MosaicId("41BE3897B2568D17"), sym.UInt64.fromUint(1))], //hossiiii.tomato
    sym.PlainMessage.create(""),
    networkType
  ).setMaxFee(100);
signedTx = alice.sign(trTx,generationHash);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

<span style="color:red">XYM以外の受信制限を解除</span>  

In [ ]:
mosaicId = new sym.MosaicId("72C0212E67A08BCE"); //テストネット XYM
tx = sym.AccountRestrictionTransaction.createMosaicRestrictionModificationTransaction(
  sym.Deadline.create(epochAdjustment),
  sym.MosaicRestrictionFlag.AllowMosaic, //モザイク制限フラグ 　速習SymbolではBlockMosaicだが勉強会の都合上AllowMosaicに変更
  [],//設定モザイク
  [mosaicId],//解除モザイク
  networkType
).setMaxFee(100);
signedTx = carol.sign(tx,generationHash);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

<span style="color:red">再度aliceからcarolへのhossiiii.tomatoの送信（制限が解除され成功することの確認）</span>  

In [ ]:
trTx = sym.TransferTransaction.create(
    sym.Deadline.create(epochAdjustment),
    carol.address, 
    [new sym.Mosaic(new sym.MosaicId("41BE3897B2568D17"), sym.UInt64.fromUint(1))], //hossiiii.tomato
    sym.PlainMessage.create(""),
    networkType
  ).setMaxFee(100);
signedTx = alice.sign(trTx,generationHash);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

### 指定トランザクションの送信制限


In [ ]:
tx = sym.AccountRestrictionTransaction.createOperationRestrictionModificationTransaction(
    sym.Deadline.create(epochAdjustment),
    sym.OperationRestrictionFlag.BlockOutgoingTransactionType, //速習SymbolではAllowOutgoingTransactionTypeだが勉強会の都合上BlockOutgoingTransactionTypeに変更
    [sym.TransactionType.AGGREGATE_COMPLETE],//AGGREGATEトランザクションを禁止
    [],//解除トランザクション
    networkType
  ).setMaxFee(100);
  signedTx = carol.sign(tx,generationHash);
  await txRepo.announce(signedTx).toPromise();  

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

OperationRestrictionFlagについては以下の通りです。
```js
{16388: 'AllowOutgoingTransactionType', 49156: 'BlockOutgoingTransactionType'}
```

- AllowOutgoingTransactionType：指定トランザクションの送信のみ許可
- BlockOutgoingTransactionType：指定トランザクションの送信を禁止

トランザクション受信の制限機能はありません。指定できるオペレーションは以下の通りです。

TransactionTypeについては以下の通りです。
```js
{16705: 'AGGREGATE_COMPLETE', 16707: 'VOTING_KEY_LINK', 16708: 'ACCOUNT_METADATA', 16712: 'HASH_LOCK', 16716: 'ACCOUNT_KEY_LINK', 16717: 'MOSAIC_DEFINITION', 16718: 'NAMESPACE_REGISTRATION', 16720: 'ACCOUNT_ADDRESS_RESTRICTION', 16721: 'MOSAIC_GLOBAL_RESTRICTION', 16722: 'SECRET_LOCK', 16724: 'TRANSFER', 16725: 'MULTISIG_ACCOUNT_MODIFICATION', 16961: 'AGGREGATE_BONDED', 16963: 'VRF_KEY_LINK', 16964: 'MOSAIC_METADATA', 16972: 'NODE_KEY_LINK', 16973: 'MOSAIC_SUPPLY_CHANGE', 16974: 'ADDRESS_ALIAS', 16976: 'ACCOUNT_MOSAIC_RESTRICTION', 16977: 'MOSAIC_ADDRESS_RESTRICTION', 16978: 'SECRET_PROOF', 17220: 'NAMESPACE_METADATA', 17229: 'MOSAIC_SUPPLY_REVOCATION', 17230: 'MOSAIC_ALIAS'}
```

##### 注意事項
17232: 'ACCOUNT_OPERATION_RESTRICTION' の制限は許可されていません。
つまり、AllowOutgoingTransactionTypeを指定する場合は、ACCOUNT_OPERATION_RESTRICTIONを必ず含める必要があり、
BlockOutgoingTransactionTypeを指定する場合は、ACCOUNT_OPERATION_RESTRICTIONを含めることはできません。

### 確認

設定した制限情報を確認します


In [ ]:
resAccountRepo = repo.createRestrictionAccountRepository();
res = await resAccountRepo.getAccountRestrictions(carol.address).toPromise();
console.log(res);

<span style="color:red">carolからbobにトランスファーTxを送ってみる（成功することの確認）</span>  

In [ ]:
trTx = sym.TransferTransaction.create(
    sym.Deadline.create(epochAdjustment),
    bob.address, 
    [new sym.Mosaic(new sym.MosaicId("72C0212E67A08BCE"), sym.UInt64.fromUint(1))],
    sym.PlainMessage.create(""),
    networkType
  ).setMaxFee(100);
signedTx = carol.sign(trTx,generationHash);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

<span style="color:red">carolからbobにアグリゲートTxを送ってみる（失敗することの確認）</span>  

In [ ]:
trTx = sym.TransferTransaction.create(
    sym.Deadline.create(epochAdjustment),
    bob.address, 
    [new sym.Mosaic(new sym.MosaicId("72C0212E67A08BCE"), sym.UInt64.fromUint(1))],
    sym.PlainMessage.create(""),
    networkType
  );
aggregateArray = [
trTx.toAggregate(carol.publicAccount),
]
aggregateTx = sym.AggregateTransaction.createComplete(
sym.Deadline.create(epochAdjustment),
aggregateArray,
networkType,[]
).setMaxFeeForAggregate(100, 1);
signedTx = carol.sign(aggregateTx,generationHash);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

<span style="color:red">carolのアグリゲートTxの制限を解除する</span>  

In [ ]:
tx = sym.AccountRestrictionTransaction.createOperationRestrictionModificationTransaction(
  sym.Deadline.create(epochAdjustment),
  sym.OperationRestrictionFlag.BlockOutgoingTransactionType, //速習SymbolではAllowOutgoingTransactionTypeだが勉強会の都合上BlockOutgoingTransactionTypeに変更
  [],
  [sym.TransactionType.AGGREGATE_COMPLETE],//AGGREGATEトランザクションを禁止を解除
  networkType
).setMaxFee(100);
signedTx = carol.sign(tx,generationHash);
await txRepo.announce(signedTx).toPromise();  

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

<span style="color:red">再度carolからbobにアグリゲートTxを送ってみる（今度は成功することの確認）</span>  

In [ ]:
trTx = sym.TransferTransaction.create(
    sym.Deadline.create(epochAdjustment),
    bob.address, 
    [new sym.Mosaic(new sym.MosaicId("72C0212E67A08BCE"), sym.UInt64.fromUint(1))],
    sym.PlainMessage.create(""),
    networkType
  );
aggregateArray = [
trTx.toAggregate(carol.publicAccount),
]
aggregateTx = sym.AggregateTransaction.createComplete(
sym.Deadline.create(epochAdjustment),
aggregateArray,
networkType,[]
).setMaxFeeForAggregate(100, 1);
signedTx = carol.sign(aggregateTx,generationHash);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

## 11.2 グローバルモザイク制限

グローバルモザイク制限はモザイクに対して送信可能な条件を設定します。  
その後、各アカウントに対してグローバルモザイク制限専用の数値メタデータを付与します。  
送信アカウント・受信アカウントの両方が条件を満たした場合のみ、該当モザイクを送信することができます。  


<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229625035-d3408a69-bb9a-40cc-b405-70fdf68ccbd2.png">

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229625053-4a91f670-e1e1-4d20-a32f-cabb082c5915.png">

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229625065-d561e4cd-aaf2-4701-a10e-cd4f7c063cb0.png">

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/229625087-5411527e-004d-45d6-ba0f-52ccb3c650f7.png">


最初に必要ライブラリの設定を行います。

In [ ]:
nsRepo = repo.createNamespaceRepository();
resMosaicRepo = repo.createRestrictionMosaicRepository();
mosaicResService = new sym.MosaicRestrictionTransactionService(resMosaicRepo,nsRepo);

### グローバル制限機能つきモザイクの作成
restrictableをtrueにしてCarolでモザイクを作成します。

In [ ]:
supplyMutable = true; //供給量変更の可否
transferable = true; //第三者への譲渡可否
restrictable = true; //グローバル制限設定の可否
revokable = true; //発行者からの還収可否
nonce = sym.MosaicNonce.createRandom();
mosaicDefTx = sym.MosaicDefinitionTransaction.create(
    undefined,
    nonce,
    sym.MosaicId.createFromNonce(nonce, carol.address),
    sym.MosaicFlags.create(supplyMutable, transferable, restrictable, revokable),
    0,//divisibility
    sym.UInt64.fromUint(0), //duration
    networkType
);
//モザイク変更
mosaicChangeTx = sym.MosaicSupplyChangeTransaction.create(
    undefined,
    mosaicDefTx.mosaicId,
    sym.MosaicSupplyChangeAction.Increase,
    sym.UInt64.fromUint(1000000),
    networkType
);
//グローバルモザイク制限
key = sym.KeyGenerator.generateUInt64Key("KYC") // restrictionKey 
mosaicGlobalResTx = await mosaicResService.createMosaicGlobalRestrictionTransaction(
    undefined,
    networkType,
    mosaicDefTx.mosaicId,
    key,
    '1',
    sym.MosaicRestrictionType.EQ,
).toPromise();
aggregateTx = sym.AggregateTransaction.createComplete(
    sym.Deadline.create(epochAdjustment),
    [
      mosaicDefTx.toAggregate(carol.publicAccount),
      mosaicChangeTx.toAggregate(carol.publicAccount),
      mosaicGlobalResTx.toAggregate(carol.publicAccount)
    ],
    networkType,[],
).setMaxFeeForAggregate(100, 0);
signedTx = carol.sign(aggregateTx,generationHash);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

MosaicRestrictionTypeについては以下の通りです。

```js
{0: 'NONE', 1: 'EQ', 2: 'NE', 3: 'LT', 4: 'LE', 5: 'GT', 6: 'GE'}
```

| 演算子  | 略称  | 英語  |
|---|---|---|
| =  | EQ  | equal to  |
| !=  | NE  | not equal to  |
| <  | LT  | less than  |
| <=  | LE  | less than or equal to  |
| >  | GT  | greater than  |
| <=  | GE  | greater than or equal to  |


### アカウントへのモザイク制限適用

Carol,Bobに対してグローバル制限モザイクに対しての適格情報を追加します。  
送信・受信についてかかる制限なので、すでに所有しているモザイクについての制限はありません。  
送信を成功させるためには、送信者・受信者双方が条件をクリアしている必要があります。  
モザイク作成者の秘密鍵があればどのアカウントに対しても承諾の署名を必要とせずに制限をつけることができます。  


In [ ]:
//Carolに適用
carolMosaicAddressResTx =  sym.MosaicAddressRestrictionTransaction.create(
    sym.Deadline.create(epochAdjustment),
    mosaicDefTx.mosaicId, // mosaicId
    sym.KeyGenerator.generateUInt64Key("KYC"), // restrictionKey
    carol.address, // address
    sym.UInt64.fromUint(1), // newRestrictionValue
    networkType,
    sym.UInt64.fromHex('FFFFFFFFFFFFFFFF') //previousRestrictionValue
).setMaxFee(100);
signedTx = carol.sign(carolMosaicAddressResTx,generationHash);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

In [ ]:
//Bobに適用
bob = sym.Account.generateNewAccount(networkType);
bobMosaicAddressResTx =  sym.MosaicAddressRestrictionTransaction.create(
    sym.Deadline.create(epochAdjustment),
    mosaicDefTx.mosaicId, // mosaicId
    sym.KeyGenerator.generateUInt64Key("KYC"), // restrictionKey
    bob.address, // address
    sym.UInt64.fromUint(1), // newRestrictionValue
    networkType,
    sym.UInt64.fromHex('FFFFFFFFFFFFFFFF') //previousRestrictionValue
).setMaxFee(100);
signedTx = carol.sign(bobMosaicAddressResTx,generationHash);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

### 制限状態確認

ノードに問い合わせて制限状態を確認します。

In [ ]:
res = await resMosaicRepo.search({mosaicId:mosaicDefTx.mosaicId}).toPromise();
console.log(res);

###### 出力例
```js
> data
    > 0: MosaicGlobalRestriction
      compositeHash: "68FBADBAFBD098C157D42A61A7D82E8AF730D3B8C3937B1088456432CDDB8373"
      entryType: 1
    > mosaicId: MosaicId
        id: Id {lower: 2467167064, higher: 973862467}
    > restrictions: Array(1)
        0: MosaicGlobalRestrictionItem
          key: UInt64 {lower: 2424036727, higher: 2165465980}
          restrictionType: 1
          restrictionValue: UInt64 {lower: 1, higher: 0}
    > 1: MosaicAddressRestriction
      compositeHash: "920BFD041B6D30C0799E06585EC5F3916489E2DDF47FF6C30C569B102DB39F4E"
      entryType: 0
    > mosaicId: MosaicId
        id: Id {lower: 2467167064, higher: 973862467}
    > restrictions: Array(1)
        0: MosaicAddressRestrictionItem
          key: UInt64 {lower: 2424036727, higher: 2165465980}
          restrictionValue: UInt64 {lower: 1, higher: 0}
          targetAddress: Address {address: 'TAZCST2RBXDSD3227Y4A6ZP3QHFUB2P7JQVRYEI', networkType: 152}
  > 2: MosaicAddressRestriction
  ...
```

### 送信確認

実際にモザイクを送信してみて、制限状態を確認します。

In [ ]:
//成功（CarolからBobに送信）
trTx = sym.TransferTransaction.create(
        sym.Deadline.create(epochAdjustment),
        bob.address, 
        [new sym.Mosaic(mosaicDefTx.mosaicId, sym.UInt64.fromUint(1))],
        sym.PlainMessage.create(""),
        networkType
      ).setMaxFee(100);
signedTx = carol.sign(trTx,generationHash);
await txRepo.announce(signedTx).toPromise();


In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

In [ ]:
//失敗（CarolからDaveに送信）
dave = sym.Account.generateNewAccount(networkType);
trTx = sym.TransferTransaction.create(
        sym.Deadline.create(epochAdjustment),
        dave.address, 
        [new sym.Mosaic(mosaicDefTx.mosaicId, sym.UInt64.fromUint(1))],
        sym.PlainMessage.create(""),
        networkType
      ).setMaxFee(100);
signedTx = carol.sign(trTx,generationHash);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

失敗した場合以下のようなエラーステータスになります。

```js
{"hash":"E3402FB7AE21A6A64838DDD0722420EC67E61206C148A73B0DFD7F8C098062FA","code":"Failure_RestrictionMosaic_Account_Unauthorized","deadline":"12371602742","group":"failed"}
```

## 11.3 現場で使えるヒント

ブロックチェーンの社会実装などを考えたときに、法律や信頼性の見地から
一つの役割のみを持たせたいアカウント、関係ないアカウントを巻き込みたくないと思うことがあります。
そんな場合にアカウント制限とグローバルモザイク制限を使いこなすことで、
モザイクのふるまいを柔軟にコントロールすることができます。

### アカウントバーン

AllowIncomingAddressによって指定アドレスからのみ受信可能にしておいて、  
XYMを全量送信すると、秘密鍵を持っていても自力では操作困難なアカウントを明示的に作成することができます。  
（最小手数料を0に設定したノードによって承認されることもあり、その可能性はゼロではありません）  

### モザイクロック
譲渡不可設定のモザイクを配布し、配布者側のアカウントで受け取り拒否を行うとモザイクをロックさせることができます。

### 所属証明
モザイクの章で所有の証明について説明しました。グローバルモザイク制限を活用することで、
KYCが済んだアカウント間でのみ所有・流通させることが可能なモザイクを作り、所有者のみが所属できる独自経済圏を構築することが可能です。